In [1]:
#hide
#default_exp export

# nbprocess.export
- Exporting a notebook to a library

In [2]:
#export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.imports import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.xtras import *

from collections import defaultdict
from pprint import pformat
from inspect import signature,Parameter
import ast,contextlib,copy

In [3]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

## NBProcessor -

Special comments at the start of a cell can be used to provide information to `nbprocess` about how to process a cell, so we need to be able to find the location of these comments.

In [4]:
minimal = read_nb('../tests/minimal.ipynb')

In [5]:
#export
def extract_comments(ss):
    "Take leading comments from lines of code in `ss`, remove `#`, and split"
    ss = ss.splitlines()
    first_code = first(i for i,o in enumerate(ss) if not o.strip() or re.match('\s*[^#\s]', o))
    return L((s.strip()[1:]).strip().split() for s in ss[:first_code]).filter()

nbprocess comments start with `#`, followed by whitespace delimited tokens, which `extract_comments` extracts from the start of a cell, up until a blank line or a line containing something other than comments:

In [6]:
exp  = """#export module
# hide
1+2
#bar"""
test_eq(extract_comments(exp), [['export', 'module'],['hide']])

In [7]:
#export
@functools.lru_cache(maxsize=None)
def _param_count(f):
    "Number of parameters accepted by function `f`"
    params = list(signature(f).parameters.values())
    # If there's a `*args` then `f` can take as many params as needed
    if first(params, lambda o: o.kind==Parameter.VAR_POSITIONAL): return 99
    return len([o for o in params if o.kind in (Parameter.POSITIONAL_ONLY,Parameter.POSITIONAL_OR_KEYWORD)])

In [8]:
#export
class NBProcessor:
    "Process cells and nbdev comments in a notebook"
    def __init__(self, path=None, procs=None, nb=None, debug=False):
        self.nb = read_nb(path) if nb is None else nb
        self.procs,self.debug = L(procs),debug

    def _process_cell(self, cell):
        self.cell = cell
        cell._comments = extract_comments(cell.source)
        for proc in self.procs:
            if callable(proc): proc(cell)
            if cell.cell_type=='code':
                for comment in cell._comments: self._process_comment(proc, comment)

    def _process_comment(self, proc, comment):
        cmd,*args = comment
        f = getattr(proc, f'_{cmd}_', None)
        if not f or _param_count(f)-1<len(args): return True
        if self.debug: print(cmd, args, f)
        return f(self, *args)

    def process(self):
        "Process all cells with `process_cell`"
        for i in range_of(self.nb.cells): self._process_cell(self.nb.cells[i])
        self.nb.cells = [c for c in self.nb.cells if c.source is not None]

In [9]:
class _PrintExample:
    def _printme_(self, nbp, to_print): print(to_print)

In [10]:
everything_fn = '../tests/01_everything.ipynb'
proc = NBProcessor(everything_fn, _PrintExample())
proc.process()

testing


## ExportModuleProc -

In [11]:
#export
class ExportModuleProc:
    "A processor which exports code to a module"
    def __init__(self): self.modules,self.in_all = defaultdict(L),defaultdict(L)
    def _default_exp_(self, nbp, exp_to): self.default_exp = exp_to
    def _exporti_(self, nbp, exp_to=None): self.modules[ifnone(exp_to, '#')].append(nbp.cell)
    def _export_(self, nbp, exp_to=None):
        self._exporti_(nbp, exp_to)
        self.in_all[ifnone(exp_to, '#')].append(nbp.cell)

Specify `dest` where the module(s) will be exported to, and optionally a class to use to create the module (`ModuleMaker`, by default).

Exported cells are stored in a `dict` called `modules`, where the keys are the modules exported to. Those without an explicit module are stored in the `'#'` key, which will be exported to `default_exp`.

In [12]:
exp = ExportModuleProc()
proc = NBProcessor(everything_fn, exp)
proc.process()
test_eq(exp.default_exp, 'everything')
assert 'print_function'  in exp.modules['#'][0].source
assert 'h_n' in exp.in_all['some.thing'][0].source

In [13]:
#export
def rm_comments_proc(cell):
    "A proc that removes comments from each NB cell source"
    cell.source = ''.join(cell.source.splitlines(True)[len(cell._comments):])

In [14]:
#export
def create_modules(path, dest, procs=None, debug=False, mod_maker=ModuleMaker):
    "Create module(s) from notebook"
    exp = ExportModuleProc()
    nb = NBProcessor(path, [exp,rm_comments_proc]+L(procs), debug=debug)
    nb.process()
    for mod,cells in exp.modules.items():
        all_cells = exp.in_all[mod]
        name = exp.default_exp if mod=='#' else mod
        mm = mod_maker(dest=dest, name=name, nb_path=path, is_new=mod=='#')
        mm.make(cells, all_cells)

Let's check we can import a test file:

In [15]:
shutil.rmtree('tmp', ignore_errors=True)
create_modules('../tests/00_some.thing.ipynb', 'tmp')

g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a'])
test_eq(g['tmp'].some.thing.a, 1)

We'll also check that our 'everything' file exports correctly:

In [16]:
create_modules(everything_fn, 'tmp')

g = exec_new('import tmp.everything; from tmp.everything import *')
_alls = L("a b d e m n o p q".split())
for s in _alls.map("{}_y"): assert s in g, s
for s in "c_y_nall _f_y_nall g_n h_n i_n j_n k_n l_n".split(): assert s not in g, s
for s in _alls.map("{}_y") + ["c_y_nall", "_f_y_nall"]: assert hasattr(g['tmp'].everything,s), s

That notebook should also export one extra function to `tmp.some.thing`:

In [17]:
del(sys.modules['tmp.some.thing']) # remove from module cache
g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a','h_n'])
test_eq(g['tmp'].some.thing.h_n(), None)

In [18]:
#export
def nb_export(nbname, lib_name=None):
    if lib_name is None: lib_name = get_config().lib_name
    create_modules(nbname, lib_name)

In [19]:
#export
@call_parse
def nbs_export(
    path:str='.', # path or filename
    recursive:bool=True, # search subfolders
    symlinks:bool=True, # follow symlinks?
    file_glob:str='*.ipynb', # Only include files matching glob
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None, # Skip files matching regex
    skip_folder_re:str='^[_.]' # Skip folders matching regex
):
    if os.environ.get('IN_TEST',0): return
    if not recursive: skip_folder_re='.'
    files = globtastic(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re,
        folder_re=folder_re, skip_file_glob=skip_file_glob, skip_file_re=skip_file_re, skip_folder_re=skip_folder_re)
    files.map(nb_export)

## Export -

In [20]:
#skip
Path('../nbprocess/export.py').unlink(missing_ok=True)
nbs_export()

g = exec_new('import nbprocess.export')
assert hasattr(g['nbprocess'].export, 'nb_export')